In [22]:
%reset
%run ../../benchmarking/utils.ipynb

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
{'gpt3': {'daily': Usage(period='2023-2-18', used=549655, quota=None), 'monthly': Usage(period='2023-2', used=549655, quota=None), 'total': Usage(period='all', used=2904655, quota=4000000)}, 'codex': {'daily': Usage(period=None, used=0, quota=10000)}, 'jurassic': {'daily': Usage(period=None, used=0, quota=10000)}, 'gooseai': {'daily': Usage(period=None, used=0, quota=10000)}, 'cohere': {'daily': Usage(period=None, used=0, quota=10000)}, 'text_to_image': {'daily': Usage(period=None, used=0, quota=10)}, 'dall_e': {'daily': Usage(period=None, used=0, quota=5)}, 'together_vision': {'daily': Usage(period=None, used=0, quota=30)}, 'gpt': {'daily': Usage(period=None, used=0, quota=None), 'monthly': Usage(period=None, used=0, quota=None)}}


In [23]:
import json
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer




In [24]:
def make_paired_dataset(df, num_samples, input_file, output_file, previous_files, question_field):
    df = pd.read_csv(input_file)
    #filter out previously used examples if any
    for x in previous_files:
        previous_file = pd.read_csv(x)
        df = df[~df[question_field].isin(previous_file[question_field])]
        df = df[~df[question_field].isin(previous_file[question_field + "_example"])]

    
    sample_questions = df.sample(n=num_samples*2).reset_index()
    sample_examples = df.sample(n=num_samples*2).reset_index()
    sample_examples.columns = [x + "_example" for x in sample_examples.columns]
    
    #create a paired dataset
    sample_questions = pd.concat([sample_questions, sample_examples], axis=1)

    #drop matching answers
    sample_questions = sample_questions[sample_questions[question_field] != sample_questions[question_field + '_example']]
    sample_questions = sample_questions.sample(n=num_samples)
    
    sample_questions.to_csv(output_file, index=False)
    return sample_questions

In [25]:
def make_paired_dataset_og(df, num_samples, output_file):
    
    sample_questions = df.sample(n=num_samples*2).reset_index()
    sample_examples = df.sample(n=num_samples*2).reset_index()
    sample_examples.columns = [x + "_example" for x in sample_examples.columns]
    
    #create a paired dataset
    sample_questions = pd.concat([sample_questions, sample_examples], axis=1)

    #drop matching answers
    sample_questions = sample_questions[sample_questions['question'] != sample_questions['question_example']]
    sample_questions = sample_questions.sample(n=num_samples)
    
    sample_questions.to_csv(output_file, index=False)
    return sample_questions

In [26]:
def filter_one_token_responses(df, answer_column_name):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    #tokenize all words using GPT2's tokenizer
    df['tokens'] = df[answer_column_name].apply(lambda x: len(tokenizer(" " + str(x))['input_ids']))
    df = df[df['tokens']==1]
    #length of single token answer
    print("Total number of one token answer examples:", len(df))
    return df

In [27]:
SAMPLE_SIZE = 100

# Trivia QA

In [28]:
def read_triviaqa_data():
    # load data using Python JSON module
    with open("triviaqa/data/web-train.json",'r') as f:
        data = json.loads(f.read())

    # Flatten data
    df = pd.json_normalize(data, record_path =['Data'])
    # basic statistics
    print("Total number of examples:", len(df))
    print(df.columns)
    df = df.dropna(subset=['Answer.MatchedWikiEntityName'])
    print("Total number of examples with non nan answers", len(df))
    df['length_of_answer'] = df['Answer.MatchedWikiEntityName'].apply(lambda x: len(x.split(" ")))
    df = df[df['length_of_answer'] == 1]
    print("Total number of one word answer examples:", len(df))
    df = filter_one_token_responses(df, 'Answer.MatchedWikiEntityName')
    df.to_csv("triviaqa/data/sample_questions_single_token_answers.csv")
    return df

In [29]:
input_file = "triviaqa/data/sample_questions_single_token_answers.csv"
output_file = "triviaqa/data/sample_questions_single_token_answers_100.csv"
previous_files = ["triviaqa/data/sample_questions_single_token_answers_30.csv", 
                  "triviaqa/data/sample_questions_single_token_answers_70.csv",
                 "triviaqa/data/sample_questions_single_token_answers_50_2.csv"]
df = read_triviaqa_data()
make_paired_dataset(df, SAMPLE_SIZE, input_file, output_file, previous_files, 'Question')

Total number of examples: 76496
Index(['EntityPages', 'Question', 'QuestionId', 'QuestionSource',
       'SearchResults', 'Answer.Aliases', 'Answer.MatchedWikiEntityName',
       'Answer.NormalizedAliases', 'Answer.NormalizedMatchedWikiEntityName',
       'Answer.NormalizedValue', 'Answer.Type', 'Answer.Value',
       'Answer.Multiplier', 'Answer.Number', 'Answer.Unit', 'Answer.From',
       'Answer.To'],
      dtype='object')
Total number of examples with non nan answers 70982
Total number of one word answer examples: 33747
Total number of one token answer examples: 17193


,index,Unnamed: 0,EntityPages,Question,QuestionId,QuestionSource,SearchResults,Answer.Aliases,Answer.MatchedWikiEntityName,Answer.NormalizedAliases,...,Answer.NormalizedValue_example,Answer.Type_example,Answer.Value_example,Answer.Multiplier_example,Answer.Number_example,Answer.Unit_example,Answer.From_example,Answer.To_example,length_of_answer_example,tokens_example
155,680,2774,"[{'DocSource': 'Search', 'Filename': 'London_U...",Which London railway station has the most plat...,qz_1454,https://www.quiz-zone.co.uk/,[{'Description': 'Which train station has the ...,"['Waterloo', 'Waterloo (film)', 'Waterloo (dis...",Waterloo,"['waterloo disambiguation', 'waterloo film', '...",...,france,WikipediaEntity,France,NaN,NaN,NaN,NaN,NaN,1,1
124,8560,33307,"[{'DocSource': 'TagMe', 'Filename': 'Gulf_of_A...",The three countries which border the Gulf of A...,qw_13014,http://www.quizwise.com/,[{'Description': 'Djibouti is a developing cou...,"['Somali National Salvation Committee', 'Somol...",Somalia,"['somaliya', 'independent somali state', 'soma...",...,cooking,WikipediaEntity,Cooking,NaN,NaN,NaN,NaN,NaN,1,1
74,2474,9545,[],Phengophobia is the irrational fear of what?,qb_6307,http://www.quizballs.com/,[{'Description': 'Phengophobia- Fear of daylig...,"['Daylight', 'Daylight (sunlight)']",Daylight,"['daylight', 'daylight sunlight']",...,spotify,WikipediaEntity,Spotify,NaN,NaN,NaN,NaN,NaN,1,1
38,11117,44187,"[{'DocSource': 'Search', 'Filename': 'United_K...",The name of which European capital translates ...,sfq_7950,www.sfquiz.org.uk,[],"['Stockholm, Sweden', 'Municipality of Stockho...",Stockholm,"['stockholm sweden', 'stochholm', 'stockohlm',...",...,mull,WikipediaEntity,MULL,NaN,NaN,NaN,NaN,NaN,1,1
149,4107,16022,"[{'DocSource': 'TagMe', 'Filename': 'House_of_...",Which country is ruled by the House of Thani?,dpql_5293,https://derbyshirepubquizleague.wordpress.com/,[{'Description': '5 countries that are governe...,"['Qtaar', 'Qatar', 'Quatar', 'Qatar Peninsula'...",Qatar,"['rumaihi', 'dawlet qaṭar', 'qatar peninsula',...",...,black,WikipediaEntity,Black,NaN,NaN,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,6579,25567,"[{'DocSource': 'TagMe', 'Filename': 'Shield.tx...",Used by ancient Greeks and Romans what is a pe...,bb_8959,http://www.businessballs.com/,[{'Description': 'A Dictionary of Greek and Ro...,"['Pelta', 'Tri-shield', 'Pelte', 'Trishield', ...",Shield,"['pelte', 'shield', 'parrying shield', 'trishi...",...,winchester,WikipediaEntity,Winchester,NaN,NaN,NaN,NaN,NaN,1,1
4,13707,58716,"[{'DocSource': 'TagMe', 'Filename': 'Michael_W...",Michael Watson won the Helen Rollason Award in...,odql_455,http://www.odquiz.org.uk/,"[{'Description': ""Don't forget to watch the Pr...","['Prize fight', 'Prize fighting', 'Fistfightin...",Boxing,"['history of boxing', 'fistfighting', 'fist fi...",...,silver,WikipediaEntity,Silver,NaN,NaN,NaN,NaN,NaN,1,1
8,1416,5963,"[{'DocSource': 'TagMe', 'Filename': 'White_on_...",Which band released a 1997 album entitled ‘Whi...,qb_1376,http://www.quizballs.com/,[{'Description': 'White on Blonde is the fourt...,"['Texas (U.S. State)', 'US-TX', 'Texos', 'Texa...",Texas,"['texas', 'religion in texas', 'texos', 'chris...",...,ghostbusters,WikipediaEntity,Ghostbusters,NaN,NaN,NaN,NaN,NaN,1,1
184,15163,67805,"[{'DocSource': 'TagMe', 'Filename': 'Daktari.t...",What is the name of the cross-eyed Lion in 'Da...,odql_14469,http://www.odquiz.org.uk/,[{'Description': '... the all-family adventure...,"['Clarence (TV series)', 'Clarence', 'Clarence...",Clarence,"['clarence tv series', 'clarence disambiguatio...",...,shells,WikipediaEntity,SHELLS,NaN,NaN,NaN,NaN,NaN,1,1


# Country QA

In [30]:
def read_country_qa_data():
    df = pd.read_csv("countryqa/data/capitals_data.csv")
    df = df[['name', 'region', 'gdp', 'gdp_logged', 'capital']]
    df['name'] = df['name'].apply(lambda x: "What is the capital of " + x + "?")
    df = filter_one_token_responses(df, 'capital')
    print("Total number of examples:", len(df))
    df = df.dropna(subset=['capital']).reset_index()
    print("Total number of examples with non nan answers", len(df))
    return df

In [31]:
def country_qa_paired_dataset(df):
    sample_examples = df.sample(frac=1, random_state=10)
    sample_examples = sample_examples.reset_index()
    sample_examples.columns = [x + "_example" for x in sample_examples.columns]
    sample_questions = pd.concat([df, sample_examples], axis=1)
    # make sure there are no overlaps between questions and examples
    display(sample_questions[sample_questions['name'] == sample_questions['name_example']])
    sample_questions.sample(n=SAMPLE_SIZE).to_csv(SAMPLE_QUESTIONS_FILE, index=False)

In [32]:
# SAMPLE_QUESTIONS_FILE = "countryqa/data/sample_questions_single_token_answers_30_2.csv"

df = read_country_qa_data()
#MAKE SURE THERE ARE NO OVERLAPS -- if so, run again
# country_qa_paired_dataset(df)

Total number of one token answer examples: 53
Total number of examples: 53
Total number of examples with non nan answers 53


# Jeopardy QA

In [33]:
def read_jeopardyqa_data():
    # load data using Python JSON module
    with open("jeopardy/data/jeopardy.json",'r') as f:
        data = json.loads(f.read())
    #preprocessing
    df = pd.json_normalize(data)
    df['value'] = df['value'].str.replace(",", "")
    df = df.dropna(subset='value')
    #removing the quotes
    df['question'] = df['question'].apply(lambda x: x[1:-1])
    print("Total number of examples:", len(df))
    df = df.dropna(subset=['answer'])
    print("Total number of examples with non nan answers", len(df))
    df = filter_one_token_responses(df, 'answer')
    df['isnumeric'] = df['answer'].apply(lambda x: x.isnumeric())
    df = df[df['isnumeric']==False]
    print("Total number of non-numeric answer examples:", len(df))
    return df

In [34]:
df = read_jeopardyqa_data()
# df.to_csv("jeopardy/data/sample_questions_single_token_answers.csv", index=False)

input_file = "jeopardy/data/sample_questions_single_token_answers.csv"
output_file = "jeopardy/data/sample_questions_single_token_answers_100.csv"
previous_files = ["jeopardy/data/sample_questions_single_token_answers_50.csv",
                 "jeopardy/data/sample_questions_single_token_answers_50_2.csv"]

make_paired_dataset(df, SAMPLE_SIZE, input_file, output_file, previous_files, 'question')

Total number of examples: 213296
Total number of examples with non nan answers 213296
Total number of one token answer examples: 49007
Total number of non-numeric answer examples: 47383


,index,category,air_date,question,value,answer,round,show_number,tokens,isnumeric,index_example,category_example,air_date_example,question_example,value_example,answer_example,round_example,show_number_example,tokens_example,isnumeric_example
152,37844,LAKES & RIVERS,1997-09-18,The Moselle River forms part of the border bet...,$1000,Luxembourg,Double Jeopardy!,2999,1,False,7263,THE SOUTHERNMOST CAPITAL CITY,2001-06-25,"London,<br />Paris,<br />Lisbon",$100,Lisbon,Jeopardy!,3886,1,False
163,17401,PICKLES,2010-05-05,"Eaten between different servings of sushi, gar...",$800,ginger,Jeopardy!,5913,1,False,15657,3-LETTER WORDS,2001-06-06,"The egg of a louse, it's something picky peopl...",$600,nit,Double Jeopardy!,3873,1,False
16,32344,POTPOURRI,1990-03-12,It's the largest state entirely east of the Mi...,$1000,Georgia,Double Jeopardy!,1281,1,False,31115,"""EPI""SODES",2004-06-02,"From the Greek for ""seizure"", it's a medical c...",$600,epilepsy,Jeopardy!,4558,1,False
142,8532,COMMON BONDS,2008-03-21,"Scarlet,<br />Dr. Johnny,<br />Saturday Night",$600,fever,Jeopardy!,5425,1,False,33288,"""S"" TOUGH",2004-07-15,"Gee, ghee is a big export from this country wh...",$800,Somalia,Double Jeopardy!,4589,1,False
170,7320,LANGUAGES,1997-07-14,"In its earliest form, this Old Testament langu...",$800,Hebrew,Double Jeopardy!,2981,1,False,13005,U.S.A.,2007-10-02,"I'd like to ""lei"" around wearing nothing but a...",$200,Hawaii,Jeopardy!,5302,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,36206,TV ROLES ON FILM,2005-01-14,"Hey old chum, Val Kilmer played this man, but ...",$200,Batman,Jeopardy!,4690,1,False,46830,BIG COUNTRY,2001-05-14,"Almost twice the size of the U.S., it's the la...",$300,Russia,Jeopardy!,3856,1,False
80,43462,STATES BY COUNTIES,2008-10-02,"Palo Alto,<br />Dallas,<br />Dubuque",$400,Iowa,Double Jeopardy!,5534,1,False,27148,"GOING ""PRO""",2009-06-17,This Titan was the father of Deucalion,$800,Prometheus,Jeopardy!,5718,1,False
188,37240,HOLLOW,2009-10-26,One of the earliest records of Chinese history...,$1200,bamboo,Double Jeopardy!,5776,1,False,7288,ALPHABETICALLY LAST,2011-03-10,Of the Teenage Mutant Ninja Turtles,$1600,Raphael,Double Jeopardy!,6104,1,False
88,10393,LAKES,2003-07-18,When Lake Bonneville evaporated thousands of y...,$200,salt,Jeopardy!,4365,1,False,1186,THE HAYES YEARS,1999-10-22,In early 1880 Wabash in this state became the ...,$600,Indiana,Double Jeopardy!,3480,1,False


# Natural QA

In [35]:
def read_naturalqa_data():
    df = pd.read_json("https://raw.githubusercontent.com/google-research-datasets/natural-questions/master/nq_open/NQ-open.train.jsonl", lines=True)
    #normalizing datasets by adding in question marks
    df['question'] = df['question'].apply(lambda x: x + "?")
    # basic statistics
    print("Total number of examples:", len(df))
    df = df.dropna(subset=['answer'])
    print("Total number of examples with non nan answers", len(df))
    df['answer'] = df['answer'].apply(lambda x: x[0])
    df = filter_one_token_responses(df, 'answer')
    df['isnumeric'] = df['answer'].apply(lambda x: x.isnumeric())
    df = df[df['isnumeric']==False]
    print("Total number of non-numeric answer examples:", len(df))
    return df

In [36]:
df = read_naturalqa_data()
input_file = "naturalqa/data/sample_questions_single_token_answers.csv"
output_file = "naturalqa/data/sample_questions_single_token_answers_100.csv"
previous_files = ["naturalqa/data/sample_questions_single_token_answers_50.csv",
                 "naturalqa/data/sample_questions_single_token_answers_50_2.csv"]

make_paired_dataset(df, SAMPLE_SIZE, input_file, output_file, previous_files, 'question')


Total number of examples: 87925
Total number of examples with non nan answers 87925
Total number of one token answer examples: 17357
Total number of non-numeric answer examples: 8255


,index,answer,question,tokens,isnumeric,index_example,answer_example,question_example,tokens_example,isnumeric_example
73,5042,Croatia,where was mama mia herewe go again filmed?,1,False,6314,thirteen,how many stars were on the original american f...,1,False
98,6438,Gold,reko diq is a mining project in pakistan for?,1,False,4917,India,where does the jungle book movie take place?,1,False
15,4278,prosperity,what does the colour green mean in china?,1,False,4605,salt,what can lower the freezing point of water?,1,False
93,1975,Philip,what did angela name her baby on the office?,1,False,5714,Japan,who did we fight against in world war 2?,1,False
133,2860,Superman,who is known as the man of steel?,1,False,5759,Denver,what cities does southwest fly to in colorado?,1,False
...,...,...,...,...,...,...,...,...,...,...
180,3853,one,how many national championships does byu have ...,1,False,2240,complete,what is the meaning of name nikhil in hindi?,1,False
60,5564,strength,what does eagle represent in nigeria coat of arm?,1,False,3771,Lori,who kills the girl from happy death day?,1,False
5,6948,Calm,description given to zero on the beaufort scale?,1,False,424,Gallery,who sang the song it's so nice to be with you?,1,False
102,1590,Tokyo,where is the summer olympics going to be next?,1,False,7107,Switzerland,where was the dictator of north korea educated?,1,False


# BoolQA

In [34]:
# def read_boolqa_data():
#     with open("boolqa/data/train.jsonl") as f:
#         df = pd.DataFrame(json.loads(line) for line in f)
#     df['question'] = df['question'].apply(lambda x: x + "?")
#     df['answer'] = df['answer'].apply(lambda x: str(x))
#     print("Total number of examples:", len(df))
#     df = df.dropna(subset=['answer'])
#     print("Total number of examples with non nan answers", len(df))
#     df = filter_one_token_responses(df, 'answer')
#     return df


In [35]:
# SAMPLE_QUESTIONS_FILE = "boolqa/data/sample_questions_single_token_answers_30_2.csv"
# df = read_boolqa_data()
# sample_questions = make_paired_dataset(df, SAMPLE_SIZE, SAMPLE_QUESTIONS_FILE)


# PopQA

In [33]:
# import datasets

# popqa = datasets.load_dataset("akariasai/PopQA")["test"]   
# df = pd.DataFrame()
# features = ['id', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'question', 'possible_answers']
# for n, row in enumerate(popqa):
#     if (n % 100 == 0):
#         print(n)
#     current = []
#     for feature in features:
#         current.append(row[feature])
#     df = pd.concat([df, pd.DataFrame([current])])

# df.columns = features
# df.to_csv("popqa.csv", index=False)
    

In [76]:
def read_popqa_data():
    df = pd.read_csv("popqa/popqa.csv")
    # # basic statistics
    print("Total number of examples:", len(df))
    df = df.dropna(subset=['possible_answers'])
    df['possible_answers'] = df['possible_answers'].apply(lambda y: [x.strip("\'").replace('"', '') for x in y.lower().strip('][').split(', ')])
    df['answers'] = df['possible_answers']
    print("Total number of examples with non nan answers", len(df))
    df['answer'] = df['answers'].apply(lambda x: x[0])
    df = filter_one_token_responses(df, 'answer')
    print("Total number of non-numeric answer examples:", len(df))
    return df

In [77]:
SAMPLE_QUESTIONS_FILE = "popqa/data/sample_questions_single_token_answers_50.csv"
df = read_popqa_data()
sample_questions = make_paired_dataset_og(df, SAMPLE_SIZE, SAMPLE_QUESTIONS_FILE)

Total number of examples: 14267
Total number of examples with non nan answers 14267
Total number of one token answer examples: 726
Total number of non-numeric answer examples: 726


# PopQA Color

In [13]:
def read_popqa_color_data():
    df = pd.read_csv("popqa/popqa.csv")
    df = df[df['prop']=='color']
    # # basic statistics
    print("Total number of examples:", len(df))
    df = df.dropna(subset=['possible_answers'])
    df['possible_answers'] = df['possible_answers'].apply(lambda y: [x.strip("\'").replace('"', '') for x in y.lower().strip('][').split(', ')])
    df['answers'] = df['possible_answers']
    print("Total number of examples with non nan answers", len(df))
    df['answer'] = df['answers'].apply(lambda x: x[0])
    df = filter_one_token_responses(df, 'answer')
    print("Total number of non-numeric answer examples:", len(df))
    return df

In [18]:
df = pd.read_csv("popqa/popqa.csv")
df[df['prop']=='color']

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
11002,830263,flag of Zimbabwe,color,red,354531,472,1036579,[],"[""red color""]",http://www.wikidata.org/entity/Q159986,http://www.wikidata.org/entity/Q3142,Flag of Zimbabwe,Red,5615,51772,What color is flag of Zimbabwe?,"[""red"", ""red color""]"
11003,1522558,AFC Bournemouth,color,red,679627,472,1036579,"[""Bournemouth"",""Bournemouth and Boscombe Athle...","[""red color""]",http://www.wikidata.org/entity/Q19568,http://www.wikidata.org/entity/Q3142,AFC Bournemouth,Red,65796,51772,What color is AFC Bournemouth?,"[""red"", ""red color""]"
11004,3157421,En Avant de Guingamp,color,red,1347920,472,1036579,"[""EA Guingamp"",""EAG"",""Guingamp""]","[""red color""]",http://www.wikidata.org/entity/Q459148,http://www.wikidata.org/entity/Q3142,En Avant Guingamp,Red,5206,51772,What color is En Avant de Guingamp?,"[""red"", ""red color""]"
11005,3346896,Arsenal Fútbol Club,color,red,1437785,472,1036579,"[""Arsenal F.C."",""Arsenal de Sarand\u00ed"",""Ars...","[""red color""]",http://www.wikidata.org/entity/Q476836,http://www.wikidata.org/entity/Q3142,Arsenal de Sarandí,Red,4555,51772,What color is Arsenal Fútbol Club?,"[""red"", ""red color""]"
11006,1425545,Aston Villa F.C.,color,sky blue,637540,472,1200681,"[""Aston Villa Football Club"",""Aston Villa FC"",...",[],http://www.wikidata.org/entity/Q18711,http://www.wikidata.org/entity/Q373160,Aston Villa F.C.,Sky blue,129519,15206,What color is Aston Villa F.C.?,"[""sky blue""]"
11007,1416925,Manchester United F.C.,color,red,633673,472,1036579,"[""Red Devils"",""Man United"",""MUFC"",""Manchester ...","[""red color""]",http://www.wikidata.org/entity/Q18656,http://www.wikidata.org/entity/Q3142,Manchester United F.C.,Red,674651,51772,What color is Manchester United F.C.?,"[""red"", ""red color""]"
11008,3483883,Fleetwood Town F.C.,color,red,1505053,472,1036579,"[""Fleetwood Town Football Club"",""Fleetwood Tow...","[""red color""]",http://www.wikidata.org/entity/Q48941,http://www.wikidata.org/entity/Q3142,Fleetwood Town F.C.,Red,18308,51772,What color is Fleetwood Town F.C.?,"[""red"", ""red color""]"
11009,1528049,Doncaster Rovers F.C.,color,red,682442,472,1036579,"[""Doncaster Rovers Football Club"",""Doncaster R...","[""red color""]",http://www.wikidata.org/entity/Q19589,http://www.wikidata.org/entity/Q3142,Doncaster Rovers F.C.,Red,15353,51772,What color is Doncaster Rovers F.C.?,"[""red"", ""red color""]"
11010,1533065,Sheffield United F.C.,color,red,685180,472,1036579,"[""Sheffield United Football Club"",""Sheffield U...","[""red color""]",http://www.wikidata.org/entity/Q19607,http://www.wikidata.org/entity/Q3142,Sheffield United F.C.,Red,53286,51772,What color is Sheffield United F.C.?,"[""red"", ""red color""]"
11011,564720,Juventus F.C.,color,white,230823,472,799997,"[""Juve"",""Juventus Football Club"",""Juventus"",""J...","[""W"",""color white"",""#FFFFFF""]",http://www.wikidata.org/entity/Q1422,http://www.wikidata.org/entity/Q23444,Juventus F.C.,White,200390,38560,What color is Juventus F.C.?,"[""white"", ""W"", ""color white"", ""#FFFFFF""]"


In [14]:
SAMPLE_QUESTIONS_FILE = "popqa_color/data/sample_questions_single_token_answers_30.csv"
df = read_popqa_color_data()
sample_questions = make_paired_dataset_og(df, min(SAMPLE_SIZE, len(df)), SAMPLE_QUESTIONS_FILE)

Total number of examples: 34
Total number of examples with non nan answers 34
Total number of one token answer examples: 32
Total number of non-numeric answer examples: 32


ValueError: Cannot take a larger sample than population when 'replace=False'